In [25]:
class TableParser:
    def __init__(self, pdf_data, template):
        self.pdf_data = pdf_data
        self.template = template

    def extract_table_data(self, table_config, page_data):
        return None

In [26]:
import json
import os
from typing import Dict, Any

template_path: str = os.path.join("src", "templates", "barclays_template.json")
pdf_data_path: str = os.path.join("src", "pdf_data", "barclays_march_2_pdf_data.json")

template: Dict[str, Any] = json.load(open(template_path))
pdf_data: Dict[str, Any] = json.load(open(pdf_data_path))

output_data = json.load(open("src/outputs/barclays_march_2_output.json"))

table_parser = TableParser(pdf_data, template)


In [27]:
table_config = template["rules"][-1]["config"]

In [28]:
coordinates = table_config["columns"][0]["coordinates"]

In [34]:
coordinates

{'top_left': {'x': 0.097, 'y': 0.488},
 'bottom_right': {'x': 0.149, 'y': 0.898}}

In [29]:
page_data = pdf_data["pages"][1]

In [30]:
page_content = page_data["content"]

In [31]:
def get_items_in_bounding_box(page_data, coordinates):
    items_in_box = []
    for item in page_data:
        bounding_box = item["bounding_box"]["decimal_coordinates"]
        if (
            bounding_box["top_left"]["x"] >= coordinates["top_left"]["x"]
            and bounding_box["top_left"]["y"] >= coordinates["top_left"]["y"]
            and bounding_box["bottom_right"]["x"]
            <= coordinates["bottom_right"]["x"]
            and bounding_box["bottom_right"]["y"]
            <= coordinates["bottom_right"]["y"]
        ):
            items_in_box.append(item)
    return items_in_box

def get_text_from_items(items):
    return " ".join([item["text"] for item in items])

def get_text_from_page(page_content, coordinates):
    items_within_coordinates = get_items_in_bounding_box(
        page_content, coordinates
    )
    return get_text_from_items(items_within_coordinates)

In [32]:
get_text_from_page(page_content, coordinates)

'01 Mar 02 Mar 06 Mar 08 Mar'

In [37]:
line_separation_y_coordinates = [0.488, 0.519, 0.671, 0.834, 0.898]
coordinates = {'top_left': {'x': 0.097, 'y': 0.488},
               'bottom_right': {'x': 0.149, 'y': 0.898}}

partitioned_box = []
for i in range(len(line_separation_y_coordinates) - 1):
    partitioned_box.append({
        'top_left': {'x': coordinates['top_left']['x'], 'y': line_separation_y_coordinates[i]},
        'bottom_right': {'x': coordinates['bottom_right']['x'], 'y': line_separation_y_coordinates[i + 1]}
    })

In [40]:
coordinates = partitioned_box[0]
get_text_from_page(page_content, coordinates)


'01 Mar'

TABLE PARSER IS THE SAME AS FORM PARSER BUT WE USE LINE SEPARATION TO GET THE BOUNDING BOXES